In [ ]:
from doctest import run_docstring_examples
from datetime import datetime
#import logging
#logging.basicConfig(level=logging.INFO)

In [ ]:
def time_solution(fun, p1, p2, solution):

    start = datetime.now()
    result = fun(p1, p2)
    dauer = datetime.now() - start

    nachricht = "Ergebnis: {} Rechenzeit: {}"
    print(nachricht.format(result, dauer))

    assert result == solution

# Kamele beladen (Rekursion)

Quelle: https://www.programmieraufgaben.ch/aufgabe/kamele-beladen/6gddr4zm

## Aufgabenstellung
Ein Kamel soll optimal beladen werden. Das Kamel kann maximal 270 kg tragen. Aktuell sind Waren mit den folgenden Gewichten zu transportieren: 5, 18, 32, 34, 45, 57, 63, 69, 94, 98 und 121 kg. Nicht alle Gepäckstücke müssen verwendet werden; die 270 kg sollen aber möglichst gut, wenn nicht sogar ganz ohne Rest beladen werden.

Schreiben Sie eine rekursive Funktion, welches die maximal mögliche Beladung eines Kamels in Abhängigketi der Gesamttragfähigkeit des Kamels und der zu transportierenden Gepäckstücke berechnet.

Tipp: Für jedes Gepäckstück aus dem Vorrat kann das Problem rekursiv vereinfacht werden. Dazu wird dessen Gewicht probehalber aufgeladen. Danach wird das beste Resultat der nun noch verfügbaren weiteren Möglichkeiten gesucht und zum probehalber aufgeladenen Gewicht addiert.

Behandeln Sie zunächst die Abbruchbedingung: der verbleibende Vorrat von nicht zu schweren Gepäckstücken ist leer.

## Rekursive Lösung

In [ ]:
def maximalgewicht(kapazitaet, vorrat):
    """
    Berechnet die maximal mögliche Beladung des Kamels in rekursiver Manier.
    
    >>> print(maximalgewicht(1, []))
    0
    >>> print(maximalgewicht(0, [1, 2, 3]))
    0
    >>> print(maximalgewicht(1, [1, 1]))
    1
    >>> print(maximalgewicht(3, [1, 2, 3]))
    3
    >>> print(maximalgewicht(5, [1, 2, 3]))
    5
    >>> print(maximalgewicht(5, [6, 1, 1, 8]))
    2

    Jedes geeignete Gepäckstück wird probehalber aufgeladen
    und zur rekursiv optimalen Lösung (auf Basis des neuen Vorrats 
    und der Restkapazität) addiert.
        
    """
    if len(vorrat) < 1 or min(vorrat) > kapazitaet:  # geeigneter Vorrat leer?
        return 0
    else:
        zuladungen = []
        for item in vorrat: # jedes Gewicht probeweise aufladen und das Maximum der restlichen, daraus resultierenden möglichen Beladungen bestimmen
            if item > kapazitaet:
                continue

            vorrat_neu = vorrat.copy()
            vorrat_neu.remove(item)

            zuladung = maximalgewicht(kapazitaet - item, vorrat_neu)
            zuladungen.append(zuladung + item)

        return max(zuladungen)


run_docstring_examples(maximalgewicht, locals())

## Rekursive Lösungen testen

Ab ca. 14 Gewichten beginnt die Aufgabe die Grenze der rekursiven Berechenbarkeit zu "knacken"!

In [ ]:
time_solution(maximalgewicht, 270, [5, 18, 32, 34, 45, 63, 69], 266)
time_solution(maximalgewicht, 270, [5, 18, 32, 34, 45, 57, 63, 69, 94, 98, 121], 270)

#time_solution(maximalgewicht, 1000, [181, 130, 128, 125, 124, 121, 104, 101, 98, 94, 69, 61, 13], 1000)

# Das dritte, grössere Beispiel mit der schlanken, rekursiven Lösung zu berechnen dauert ... und dauert ... und dauert.
# Da benötigen wir wohl einen besseren Algorithmus.

## Alternative Rekursive Lösung

Diese etwas kompliziertere rekursive Variante gibt die Gewichte aus, die zur maximalen Ladung führen - ist aber auch nicht für grössere Probleme geeignet. 

In [ ]:
def aufladen(kapazitaet, vorrat, aktuelle_beladung=[]):
    """
    Berechnet rekursiv die maximale Ladung des Kamels und gibt die benötigten Gewichte als List aus.
    
    >>> print(aufladen(1, []))
    []
    >>> print(aufladen(0, [1, 2, 3]))
    []
    >>> print(aufladen(2, [1, 1]))
    [1, 1]
    >>> print(aufladen(1, [1, 1], [1]))
    [1, 1]
    >>> print(aufladen(3, [1, 2, 3]))
    [1, 2]
    >>> print(aufladen(5, [1, 2, 3]))
    [2, 3]
    >>> print(aufladen(5, [6, 1, 1, 8]))
    [1, 1]
    
    """
    #logging.info("{} {} {}".format(kapazitaet, vorrat, aktuelle_beladung))

    if len(vorrat) < 1 or min(vorrat) > kapazitaet:  # geeigneter Vorrat leer?
        return aktuelle_beladung
    else:
        max_zuladung = []   # Liste von Gewichten
        for item in vorrat: # jedes Gewicht probeweise aufladen und das Maximum der restlichen, daraus resultierenden möglichen Beladungen bestimmen
            if item > kapazitaet:
                continue

            vorrat_neu = vorrat.copy()
            vorrat_neu.remove(item)

            beladung_neu = aktuelle_beladung.copy()
            beladung_neu.append(item)

            zuladung = aufladen(kapazitaet - item, vorrat_neu, beladung_neu)

            if sum(zuladung) > sum(max_zuladung):
                max_zuladung = zuladung

        return max_zuladung


run_docstring_examples(aufladen, locals())

## Alternative Rekursive Lösungen testen

In [ ]:
time_solution(aufladen, 270, [5, 18, 32, 34, 45, 63, 69], [5, 18, 32, 34, 45, 63, 69])
time_solution(aufladen, 270, [5, 18, 32, 34, 45, 57, 63, 69, 94, 98, 121], [5, 18, 32, 94, 121])

## Dynamic Programming - Lösung
Die Standardlösung des sogenannten [Rucksackproblems](https://de.wikipedia.org/wiki/Rucksackproblem) mittels Dynamic Programming weist eine bessere Laufzeit als die rekursiven Varianten aus. 

In [ ]:
def beladen_dp(kapazitaet, vorrat):
    """
    Berechnet die maximal mögliche Beladung des Kamels mit dem Standard-DP-Algorithmus für das Rucksack-Problem mit ganzzahligen Gewichten.
    
    >>> print(beladen_dp(1, []))
    0
    >>> print(beladen_dp(0, [1, 2, 3]))
    0
    >>> print(beladen_dp(1, [1, 1]))
    1
    >>> print(beladen_dp(3, [1, 2, 3]))
    3
    >>> print(beladen_dp(5, [1, 2, 3]))
    5
    >>> print(beladen_dp(5, [6, 1, 1, 8]))
    2
    
    """
    if len(vorrat) < 1 or min(vorrat) > kapazitaet:  # geeigneter Vorrat leer?
        return 0

    # DP Tabelle vorbereiten
    # Die Werte der ersten Zeile und ersten Spalte der Matrix sind 0,
    # was den Zuständen "keine Gepäckstücke vorhanden" bzw. "keine Kapazität vorhanden" entspricht.
    dp = [[0 for j in range(kapazitaet + 1)]]  # die erste Zeile mit Nullen wird gleich beim Initialisieren mit eingefügt
    for zeile in range(len(vorrat)):           # restlichen Zeilen hinzufügen (pro Gepäckstück eine)
        row = [0]                              # der erste Wert jeder Zeile ist immer 0
        row.extend([None for j in range(kapazitaet)])
        dp.append(row)

    # DP Tabelle ausfüllen, zeilenweise, wir starten mit dem 2. Feld in der 2. Zeile
    zeilen = range(1, len(dp))
    spalten = range(1, len(dp[0]))
    for zeile in zeilen:
        item_gewicht = vorrat[zeile - 1]
        for spalte in spalten:
            spalte_ohne_item = spalte - item_gewicht  # diese Spalte repräsentiert das maximales Gewicht des Rucksacks ohne dieses Item
            zeile_ohne_item = zeile - 1
            gewicht_ohne_item = dp[zeile_ohne_item][spalte]
            if spalte_ohne_item >= 0:                 # passt das item überhaupt in den bisherigen Rucksack?
                gewicht_mit_item = dp[zeile_ohne_item][spalte_ohne_item] + item_gewicht
                dp[zeile][spalte] = max(gewicht_mit_item, gewicht_ohne_item)
            else:
                dp[zeile][spalte] = gewicht_ohne_item

    return dp[-1][-1]


run_docstring_examples(beladen_dp, locals())

## DP Lösungen testen

In [ ]:
time_solution(beladen_dp, 
              270, 
              [5, 18, 32, 34, 45, 57, 63, 69, 94, 98, 121],
              270)
time_solution(beladen_dp, 
              1000,
              [181, 130, 128, 125, 124, 121, 104, 101, 98, 94, 69, 61, 13],
              1000)